# Résolution de Sudoku par Algorithme Génétique

## Introduction Théorique

Les algorithmes génétiques (GA) sont des techniques de recherche heuristiques inspirées par le processus de sélection naturelle. Ils sont couramment utilisés pour résoudre des problèmes d'optimisation et de recherche. Un algorithme génétique utilise des opérations telles que la mutation, le croisement et la sélection pour évoluer vers une solution optimale.

Dans le contexte du Sudoku, un algorithme génétique peut être utilisé pour trouver une solution en représentant une grille de Sudoku comme un chromosome, où chaque gène représente une cellule de la grille. Les opérations génétiques sont appliquées pour optimiser la grille en respectant les contraintes du Sudoku.

## Installation de GeneticSharp

Nous devons commencer par installer le package GeneticSharp.
Afin de concevoir un solver de Sudoku en algorithme génétique, il nous faudra concevoir un chromosome de Sudoku représentant l'individu d'une population de solution potentielles, et une fonction d'évaluation (fitness) évaluant la qualité d'un individu/chromosme.
GeneticSharp fournira tout le reste des composants nécessaires à la mise en oeuvre de l'algorithme.


In [1]:
#r "nuget: GeneticSharp"

Installed Packages GeneticSharp, 3.1.4

## Importation des Classes de Base

Nous allons importer les classes de base définies dans le notebook précédent, fournissant notamment la représentation, le chargement et l'affichage de Sudokus, et l'infrastructure de résolution.


In [2]:
#!import Sudoku-0-Environment.ipynb

# Notebook 0: Classes de Base pour la Résolution de Sudoku

Ce notebook contient les classes de base nécessaires pour la manipulation et la résolution des grilles de Sudoku. Il sera importé dans les autres notebooks pour réutiliser ces classes.

## Importation des Bibliothèques Nécessaires

Nous commençons par importer les bibliothèques nécessaires.


## Définition de la classe SudokuGrid

Nous définissons ici la classe SudokuGrid qui représente une grille de Sudoku et fournit des méthodes pour manipuler et afficher les grilles.


## Définition de l'interface ISudokuSolver

Nous définissons ici l'interface ISudokuSolver qui sera implémentée par les différentes stratégies de résolution de Sudoku.


## Définition de la classe SudokuHelper

Nous ajoutons ici la classe SudokuHelper qui contient des méthodes utilitaires pour charger et afficher des grilles de Sudoku.


## Affichage des Puzzles de chaque Difficulté

Nous allons charger et afficher un puzzle de chaque niveau de difficulté : Facile, Moyen et Difficile.


Puzzle Facile:
-------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Puzzle Moyen:
-------------------------------
| 8  5    |       2 | 4       | 
| 7  2    |         |       9 | 
|       4 |         |         | 
-------------------------------
|         | 1     7 |       2 | 
| 3     5 |         | 9       | 
|    4    |         |         | 
-------------------------------
|         |    8    |    7    | 
|    1  7 |         |         | 
|         |    3  6 |    4    | 
-------------------------------

Puzzle Difficile:
-------------------------------
| 4       |         | 8     5 | 
|    3    |         |         | 
|         | 7       |         | 
-------------------------------
|    2    |         |    6    | 
|         |    8    | 4       | 
|         |    1    |         | 
-------------------------------
|         | 6     3 |    7    | 
| 5       | 2       |         | 
| 1     4 |         |         | 
-------------------------------

## Implémentation du Solveur de Sudoku par Algorithme Génétique

Nous allons maintenant implémenter ce solveur en C#. Comme indiqué précédemment il nous faut code la notion de chromosome et de fonction d'évaluation.

Afin de nous donner la possibilité de tester plusieurs types de chromosome, on introduit une interface dédiée:

### Interface `ISudokuChromosome`


In [3]:
using GeneticSharp;

public interface ISudokuChromosome: IChromosome
{
	SudokuGrid TargetSudoku { get; set; }
    SudokuGrid GetSolution();
    
}

### Classe `SudokuFitness`

On peut maintenant proposer une fonction d'évaluation basée sur cette interface.
Cette classe évalue un chromosome en fonction du nombre d'erreurs dans la grille de Sudoku générée.


In [4]:
using GeneticSharp;

public class SudokuFitness : IFitness
{
    private SudokuGrid _targetSudokuGrid;

    public SudokuFitness(SudokuGrid targetSudokuGrid)
    {
        _targetSudokuGrid = targetSudokuGrid;
    }

    public double Evaluate(IChromosome chromosome)
    {
        var sudokuChromosome = (ISudokuChromosome)chromosome;
        var sudoku = sudokuChromosome.GetSolution();
        var nbErrors = sudoku.NbErrors(_targetSudokuGrid);
        return -nbErrors;
    }
}


### Classe `SudokuSolver`

Cette classe contient les méthodes pour résoudre le Sudoku en utilisant l'algorithme génétique.

Nous concevons dores et déjà une méthode de résolution qui supportera plusieurs types de chromosomes de Sudoku sur les bases de notre interface commune, et présentant des fonctionnalités avancées suivantes:

- Possibilité de fournir l'opérateur de croisement et de mutation (opérateurs uniformes par défaut)

- Utilisation du parallélisme. L'application des opérateurs génétiques à une population ainsi que l'évaluation de la population résultantes sont par nature parallélisables. GeneticSharp propose l'utilisation du parallélisme de tâche natif .Net pour l'utilisation optimale des coeurs de processeurs disponibles.

- Multi-Objectifs et ajustement de la taille de la population. L'algorithme se termine à l'issue d'une génération si le meilleur chromosome ne contient plus d'erreur, ou si sa fitness n'a pas évolué depuis un certain nombre de génération, signalant un possible effondrement de la diversité génétique sur un maximum local. Dans ce cas, l'algorithme est redémarré en doublant la taille de la population.   


In [16]:
using GeneticSharp;
using Microsoft.DotNet.Interactive;
using System;
using System.Diagnostics;

public class SudokuGeneticSolver: ISudokuSolver
{
    
    public ISudokuChromosome Chromosome { get; set; }
    public IMutation Mutation { get; set; } = new UniformMutation(true);
    public ICrossover Crossover { get; set; } = new UniformCrossover(0.5f);


    public SudokuGeneticSolver(ISudokuChromosome chromosome)
    {
        Chromosome = chromosome;
    }

    

    public SudokuGrid Solve(SudokuGrid s)
    {
        var maxDuration = 30;
        var maxPopulation = 100000;
        var fitness = new SudokuFitness(s);
        
        var populationSize = 400;
        //Opérateur de sélection (Elite rapide mais un peu sélectif)
        var selection = new EliteSelection ();

        // Critères de terminaison
        var fitnessThreshold = 0;
        int stableGenerationNb = 30;
        var termination = new OrTermination(new ITermination[]
        {
            new FitnessThresholdTermination(fitnessThreshold),
            new FitnessStagnationTermination(stableGenerationNb),
        });
        
        var stopWatch = Stopwatch.StartNew();
        var lastTime = stopWatch.Elapsed;
        DisplayedValue displayPlaceholder = null;

        SudokuGrid bestSudoku = null;
        do
        {
            Population population = new Population(populationSize, populationSize, Chromosome);

            GeneticAlgorithm ga = new GeneticAlgorithm(population, fitness, selection, Crossover, Mutation)
            {
                Termination = termination,
                // Opérateurs de parallélisation
                OperatorsStrategy = new TplOperatorsStrategy(),
                TaskExecutor = new TplTaskExecutor(),
                //MutationProbability = 0.1f,
                // CrossoverProbability = 0.75f
            };
            
            ga.GenerationRan += (sender, args) =>
            {
                var bestIndividual = (ISudokuChromosome)ga.Population.BestChromosome;
                bestSudoku = bestIndividual.GetSolution();
                var nbErrors = bestSudoku.NbErrors(s);
                var message = $"Generation {ga.GenerationsNumber}, Population {ga.Population.CurrentGeneration.Chromosomes.Count}, NbErrors {bestSudoku.NbErrors(s)}, Elapsed {stopWatch.Elapsed - lastTime}";
                if (displayPlaceholder == null)
                    displayPlaceholder = display(message);
                else
                    displayPlaceholder.Update(message);
            
            };
            lastTime = stopWatch.Elapsed;
            ga.Start();
            populationSize = Math.Min(maxPopulation, populationSize *= 2);
        } while (bestSudoku.NbErrors(s) > 0 && stopWatch.Elapsed.TotalSeconds < maxDuration);

        return bestSudoku;

    }
    
}

### Premier chromosome simple

Pour ce premier essai, nous représentont chaque cellule par un gène à valeur de 1 à 9. L'initialisation tient compte du masque à résoudre, mais pas les opérateurs qui agissent au hasard.

In [6]:
using System;

public class SudokuCellsChromosome : ChromosomeBase, ISudokuChromosome
{
    public SudokuGrid TargetSudoku { get; set; }

    
    public SudokuCellsChromosome() : base(81) {}

    public override Gene GenerateGene(int geneIndex)
    {
        var row = geneIndex / 9;
        var col = geneIndex % 9;

        if (TargetSudoku.Cells[row, col] != 0)
        {
            return new Gene(TargetSudoku.Cells[row, col]);
        }

        var rnd = RandomizationProvider.Current;
        return new Gene(rnd.GetInt(1, 10));
    }

    public override IChromosome CreateNew()
    {
        var toReturn = new SudokuCellsChromosome(){TargetSudoku = TargetSudoku};
        toReturn.CreateGenes();
        return toReturn;
    }

    public SudokuGrid GetSolution()
    {
        var genes = GetGenes();
        var cells = new int[9, 9];
        for (int i = 0; i < 81; i++)
        {
            cells[i / 9, i % 9] = (int)genes[i].Value;
        }
        var sudoku = new SudokuGrid() { Cells = cells };
        return sudoku;
    }
}


## Test du Solveur

Nous allons maintenant tester notre solveur de Sudoku par algorithme génétique en utilisant des grilles de Sudoku de différentes difficultés : facile, moyen et difficile.


In [17]:
display("Test du solver genetique simple:");


display("Puzzle Sudoku Facile Initial:");

// Charger et tester un puzzle facile
var easySudokus = SudokuHelper.GetSudokus(SudokuDifficulty.Easy);
var easySudoku = easySudokus.FirstOrDefault();

//Création du chromosome:
var chromosome = new SudokuCellsChromosome(){TargetSudoku = easySudoku};

// Instanciation de Solver avec notre chromosmome SudokuCellsChromosome
var solver = new SudokuGeneticSolver(chromosome);

SudokuHelper.SolveSudoku(easySudoku, solver);

Test du solver genetique simple:

Puzzle Sudoku Facile Initial:

Résolution par le solver SudokuGeneticSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Generation 43, Population 1600, NbErrors 0, Elapsed 00:00:00.5027315

Sudoku renvoyé:
-------------------------------
| 9  6  2 | 1  8  5 | 4  7  3 | 
| 1  7  4 | 9  6  3 | 8  2  5 | 
| 5  3  8 | 4  2  7 | 1  6  9 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 3  5  7 | 8  1  4 | 2  9  6 | 
| 4  9  1 | 6  7  2 | 5  3  8 | 
-------------------------------
| 2  4  9 | 5  3  8 | 6  1  7 | 
| 7  1  5 | 2  9  6 | 3  8  4 | 
| 6  8  3 | 7  4  1 | 9  5  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 1574,7194 ms

## Conclusion Intermédiaire

Le solveur génétique a réussi à résoudre le Sudoku de difficulté facile en plusieurs redémarrages, prenant plusieurs secondes. Cela montre que notre approche fonctionne, mais elle n'est pas encore assez efficace pour résoudre des puzzles de difficulté plus difficile en un temps raisonnable.


### Chromosome : SudokuPermutationsChromosome

Le SudokuPermutationsChromosome est un chromosome simple de 9 gènes qui manipule les permutations de lignes du Sudoku. Chaque gène représente une permutation d'une ligne entière, permettant ainsi une exploration plus efficace des solutions potentielles.


In [18]:
public class SudokuPermutationsChromosome  : ChromosomeBase, ISudokuChromosome
{

     public SudokuGrid TargetSudoku { get; set; }
    protected static IList<IList<int>> allPermutations = GetPermutations(Enumerable.Range(1, 9), 9).ToList();

    private readonly IList<IList<int>>[] _rowPermutationsCache;

    public SudokuPermutationsChromosome(SudokuGrid targetSudoku) 
        : base(9)
    {
        TargetSudoku = targetSudoku;
        _rowPermutationsCache = new IList<IList<int>>[9];
        CreateGenes();
    }

    private SudokuPermutationsChromosome(SudokuGrid targetSudoku, IList<IList<int>>[] rowPermutationsCache)
        : base( 9)
    {
         TargetSudoku = targetSudoku;
        _rowPermutationsCache = rowPermutationsCache;
        CreateGenes();
    }

    public override Gene GenerateGene(int geneIndex)
    {
        var rowPermutations = GetRowPermutations(geneIndex);
        var rnd = RandomizationProvider.Current;
        var chosenPermutation = rowPermutations[rnd.GetInt(0, rowPermutations.Count)];
        return new Gene(chosenPermutation);
    }

    private IList<IList<int>> GetRowPermutations(int row)
    {
        if (_rowPermutationsCache[row] == null)
        {
            _rowPermutationsCache[row] = allPermutations
                .Where(perm => IsValidPermutation(perm, row)).ToList();
        }
        return _rowPermutationsCache[row];
    }

    private static IList<IList<int>> GetPermutations(IEnumerable<int> list, int length)
    {
        if (length == 1) return list.Select(t => (IList<int>)(new List<int> { t })).ToList();
        return GetPermutations(list, length - 1)
            .SelectMany(t => list.Where(e => !t.Contains(e)),
                (t1, t2) => (IList<int>)t1.Concat(new List<int> { t2 }).ToList()).ToList();
    }

    private bool IsValidPermutation(IList<int> permutation, int row)
    {
        for (int col = 0; col < 9; col++)
        {
            if (TargetSudoku.Cells[row, col] != 0 && TargetSudoku.Cells[row, col] != permutation[col])
            {
                return false;
            }
        }
        return true;
    }

    public override IChromosome CreateNew()
    {
        return new SudokuPermutationsChromosome(TargetSudoku, _rowPermutationsCache);
    }

    public SudokuGrid GetSolution()
    {
        var newGrid = (SudokuGrid)TargetSudoku.Clone();
        var genes = GetGenes();
        for (int row = 0; row < 9; row++)
        {
            var permutation = (IList<int>)genes[row].Value;
            for (int col = 0; col < 9; col++)
            {
                newGrid.Cells[row, col] = permutation[col];
            }
        }
        return newGrid;
    }
}


### Test du SudokuPermutationsChromosome

Nous allons maintenant tester le SudokuPermutationsChromosome en utilisant notre algorithme génétique. Nous allons voir comment il se comporte sur un Sudoku de difficulté facile et moyenne.

In [19]:
display("Test du solver genetique de permutations de lignes:");


display("Puzzle Sudoku Facile Initial:");

// Charger et tester un puzzle facile
var easySudokus = SudokuHelper.GetSudokus(SudokuDifficulty.Easy);
var easySudoku = easySudokus.FirstOrDefault();

//Création du chromosome:
var chromosome = new SudokuPermutationsChromosome(easySudoku);

// Instanciation de Solver avec notre chromosmome SudokuCellsChromosome
var solver = new SudokuGeneticSolver(chromosome);

SudokuHelper.SolveSudoku(easySudoku, solver);

display("Puzzle Sudoku Medium Initial:");

// Charger et tester un puzzle moyen
var mediumSudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Medium).First();

chromosome = new SudokuPermutationsChromosome(mediumSudoku);
solver = new SudokuGeneticSolver(chromosome);

SudokuHelper.SolveSudoku(mediumSudoku, solver);

Test du solver genetique de permutations de lignes:

Puzzle Sudoku Facile Initial:

Résolution par le solver SudokuGeneticSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Generation 16, Population 400, NbErrors 0, Elapsed 00:00:00.0594650

Sudoku renvoyé:
-------------------------------
| 9  6  2 | 1  8  5 | 4  7  3 | 
| 1  7  4 | 9  6  3 | 8  2  5 | 
| 5  3  8 | 4  2  7 | 1  6  9 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 3  5  7 | 8  1  4 | 2  9  6 | 
| 4  9  1 | 6  7  2 | 5  3  8 | 
-------------------------------
| 2  4  9 | 5  3  8 | 6  1  7 | 
| 7  1  5 | 2  9  6 | 3  8  4 | 
| 6  8  3 | 7  4  1 | 9  5  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 59,8228 ms

Puzzle Sudoku Medium Initial:

Résolution par le solver SudokuGeneticSolver du Sudoku:
 -------------------------------
| 8  5    |       2 | 4       | 
| 7  2    |         |       9 | 
|       4 |         |         | 
-------------------------------
|         | 1     7 |       2 | 
| 3     5 |         | 9       | 
|    4    |         |         | 
-------------------------------
|         |    8    |    7    | 
|    1  7 |         |         | 
|         |    3  6 |    4    | 
-------------------------------

Generation 95, Population 51200, NbErrors 5, Elapsed 00:00:24.6079982

Sudoku renvoyé:
-------------------------------
| 8  5  9 | 6  7  2 | 4  3  1 | 
| 7  2  6 | 4  1  3 | 5  8  9 | 
| 1  3  4 | 8  9  5 | 7  2  6 | 
-------------------------------
| 9  8  6 | 1  4  7 | 3  5  2 | 
| 3  7  5 | 2  6  8 | 9  1  4 | 
| 1  4  2 | 3  5  9 | 8  6  7 | 
-------------------------------
| 2  4  3 | 9  8  1 | 6  7  5 | 
| 6  1  7 | 5  2  4 | 3  9  8 | 
| 5  9  8 | 7  3  6 | 2  4  1 | 
-------------------------------
Nombre d'erreurs réstantes: 5
Temps de résolution: 49358,8657 ms

## Conclusion Finale

Bien que notre approche avec le SudokuPermutationsChromosome montre une amélioration par rapport au chromosome simple, il reste des défis importants pour résoudre des puzzles de difficulté moyenne et élevée. 

Le principal problème rencontré semble être un effondrement de la diversité génétique, même avec l'utilisation des permutations. Le problème du Sudoku est difficile d'accès aux algorithmes génétiques du fait notamment que l'espace de solution présente de nombreux extrema locaux sur lesquels ils restent coincés, et une très faible densité de solution.

Dans les notebooks suivants, nous explorerons d'autres méthodes et approches pour résoudre les puzzles de Sudoku, en espérant trouver des solutions plus efficaces et rapides.

### Pour aller plus loin

Pour améliorer l'efficacité des algorithmes génétiques dans la résolution de Sudoku, il serait essentiel d'explorer des opérateurs plus performants que les croisements et mutations uniformes actuellement utilisés. 

Des opérateurs dits "ordered" issus de la littérature génétique, pourraient offrir des gains significatifs. Ces opérateurs, basés sur des découpages et recollages inspirés de la génétique, permettent des transformations de type permutations sur des gènes présentés comme une liste réordonnable. Utilisés dans le problème du voyageur de commerce (TSP), ils améliorent sensiblement les qualités de la convergence.

 Appliqués au Sudoku, ils pourraient permettre la recombinaison de "bouts de bonnes solutions" pour converger plus efficacement vers la solution optimale. Mais cela impliquerait tout d'abord de géométriser le chromosome du Sudoku pour permettre leur bon fonctionnement.